# Exploring neighborhoods in Toronto 

Coursera Capstone project

In [157]:
#get dependencies
import pandas as pd
import numpy as np


In [158]:
#scrape wikipedia page to get initial table
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

tables = pd.read_html(url)
neighborhoods = tables[0]
print(neighborhoods.shape) #total neighborhoods before removing "not assigned"
neighborhoods = neighborhoods[neighborhoods['Borough']!='Not assigned']
print(neighborhoods.shape) #total neighborhoods after removing "not assigned "
print(len(neighborhoods[neighborhoods['Neighbourhood']=='Not assigned'])) #Are there any unassigned neighborhoods? No
neighborhoods.head(12)

(180, 3)
(103, 3)
0


Postal Code           Borough  \
2          M3A        North York   
3          M4A        North York   
4          M5A  Downtown Toronto   
5          M6A        North York   
6          M7A  Downtown Toronto   
8          M9A         Etobicoke   
9          M1B       Scarborough   
11         M3B        North York   
12         M4B         East York   
13         M5B  Downtown Toronto   
14         M6B        North York   
17         M9B         Etobicoke   

                                        Neighbourhood  
2                                           Parkwoods  
3                                    Victoria Village  
4                           Regent Park, Harbourfront  
5                    Lawrence Manor, Lawrence Heights  
6         Queen's Park, Ontario Provincial Government  
8             Islington Avenue, Humber Valley Village  
9                                      Malvern, Rouge  
11                                          Don Mills  
12                    Parkview Hill, Woodbine Gardens  
13                           Garden District, Ryerson  
14                                          Glencairn  
17  West Deane Park, Princess Gardens, Martin Grov...

In [159]:
#printing number of rows of the dataframe
print(neighborhoods.shape[0])

103


In [160]:
#import geosptatial coordinate csv
geo = pd.read_csv("Geospatial_Coordinates.csv", index_col=0)
geo.rename_axis(None,inplace=True)
geo.head()


Latitude  Longitude
M1B  43.806686 -79.194353
M1C  43.784535 -79.160497
M1E  43.763573 -79.188711
M1G  43.770992 -79.216917
M1H  43.773136 -79.239476

In [161]:
#make new table with lats and longs of each postal code
lats = []
longs = []

for p in neighborhoods['Postal Code']:
    lat = geo.loc[p,'Latitude'] 
    long = geo.loc[p,'Longitude'] 
    lats.append(lat)
    longs.append(long)

neighborhoods['Latitude'] = lats
neighborhoods['Longitude'] = longs

neighborhoods.head()

<ipython-input-161-c21a5837a9f4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neighborhoods['Latitude'] = lats
<ipython-input-161-c21a5837a9f4>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neighborhoods['Longitude'] = longs


Postal Code           Borough                                Neighbourhood  \
2         M3A        North York                                    Parkwoods   
3         M4A        North York                             Victoria Village   
4         M5A  Downtown Toronto                    Regent Park, Harbourfront   
5         M6A        North York             Lawrence Manor, Lawrence Heights   
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
2  43.753259 -79.329656  
3  43.725882 -79.315572  
4  43.654260 -79.360636  
5  43.718518 -79.464763  
6  43.662301 -79.389494

Here, I will begin analysis of the different neighborhoods

In [162]:
#import dependencies for this section

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


First, I will make a map of Toronto and mark the neighborhoods

In [163]:
#create a map of toronto

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [164]:
#create map of toronto area and mark the neighborhoods
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=9.5)

# add markers to map for each neighborhood
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Then, I will use Foursquare to get venue information for the different neighborhoods

In [185]:
#Define Foursquare version and credentials
CLIENT_ID = 'D2LD4G4GK304Y4SZG3DKAHPSIO1K2G4UQPIPV3LFOCDHJGIT' # your Foursquare ID
CLIENT_SECRET = 'NBJQVVS3MFHPYJOM4GE34M1IZYUZ53L25PUGNQSLY1AFXVCF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: D2LD4G4GK304Y4SZG3DKAHPSIO1K2G4UQPIPV3LFOCDHJGIT
CLIENT_SECRET:NBJQVVS3MFHPYJOM4GE34M1IZYUZ53L25PUGNQSLY1AFXVCF


In [186]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(neighborhoods['Neighbourhood'],neighborhoods['Latitude'], neighborhoods['Longitude'])



Parkwoods


KeyError: 'groups'

note, when over quota, the error above occurs. I got the error when re-running the assignment. I was able to finish the assignment before going over quota

Then, I will process the data so it can be clustered. I will only look at the top 10 venue types across all of Toronto for this analysis

In [206]:
#create new data frame with one hot encoding so we can analyze with clustering
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
toronto_onehot.head()


Accessories Store  Adult Boutique  Airport  Airport Food Court  \
0                  0               0        0                   0   
1                  0               0        0                   0   
2                  0               0        0                   0   
3                  0               0        0                   0   
4                  0               0        0                   0   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  \
0             0               0                0                 0   
1             0               0                0                 0   
2             0               0                0                 0   
3             0               0                0                 0   
4             0               0                0                 0   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                    0             0         0            0           0   
1                    0             0         0            0           0   
2                    0             0         0            0           0   
3                    0             0         0            0           0   
4                    0             0         0            0           0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Garage  \
0                    0                 0                   0            0   
1                    0                 0                   0            0   
2                    0                 0                   0            0   
3                    0                 0                   0            0   
4                    0                 0                   0            0   

   Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  \
0              0          0           0           0       0     0    0   
1              0          0           0           0       0     0    0   
2              0          0           0           0       0     0    0   
3              0          0           0           0       0     0    0   
4              0          0           0           0       0     0    0   

   Baseball Field  Baseball Stadium  Basketball Court  Basketball Stadium  \
0               0                 0                 0                   0   
1               0                 0                 0                   0   
2               0                 0                 0                   0   
3               0                 0                 0                   0   
4               0                 0                 0                   0   

   Beach  Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  \
0      0                0         0           0                   0   
1      0                0         0           0                   0   
2      0                0         0           0                   0   
3      0                0         0           0                   0   
4      0                0         0           0                   0   

   Bike Shop  Bistro  Board Shop  Boat or Ferry  Bookstore  Boutique  \
0          0       0           0              0          0         0   
1          0       0           0              0          0         0   
2          0       0           0              0          0         0   
3          0       0           0              0          0         0   
4          0       0           0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bridal Shop  \
0                     0               0        0            0   
1                     0               0        0            0   
2                     0               0        0            0   
3                     0               0        0            0   
4                     0               0        0            0   

   Bubble Tea Shop  Building  Burger Joint  Burrito Place  Bus Line  \
0                0     

In [207]:
#group by neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped.head()
print(toronto_grouped.shape)

#filter table for only most frequent venues
toronto_grouped.set_index('Neighborhood', inplace = True)
toronto_groupedT = toronto_grouped.transpose()

toronto_groupedT['sum'] = toronto_groupedT.sum(axis=1) #find total freq of of venues
toronto_groupedT.sort_values(by='sum', ascending=False, inplace=True)
#pd.set_option("display.max_rows", None, "display.max_columns", None)

toronto_filterT = toronto_groupedT[toronto_groupedT['sum']>=1.7] #filter down to only venues with high frequencies ( i.e. > 1)
print(toronto_groupedT.shape)
print(toronto_filterT.shape)
# this looks at top 10 values

#create new dataframe for clustering
toronto_top_venues = toronto_filterT.iloc[:,:-1].transpose() #remove sum column first
toronto_top_venues.head()

(94, 275)
(274, 95)
(10, 95)


Park  Coffee Shop  \
Neighborhood                                                         
Agincourt                                         0.0     0.000000   
Alderwood, Long Branch                            0.0     0.142857   
Bathurst Manor, Wilson Heights, Downsview North   0.0     0.100000   
Bayview Village                                   0.0     0.000000   
Bedford Park, Lawrence Manor East                 0.0     0.083333   

                                                 Pizza Place      Café  \
Neighborhood                                                             
Agincourt                                           0.000000  0.000000   
Alderwood, Long Branch                              0.285714  0.000000   
Bathurst Manor, Wilson Heights, Downsview North     0.050000  0.000000   
Bayview Village                                     0.000000  0.250000   
Bedford Park, Lawrence Manor East                   0.041667  0.041667   

                                                 Fast Food Restaurant  \
Neighborhood                                                            
Agincourt                                                         0.0   
Alderwood, Long Branch                                            0.0   
Bathurst Manor, Wilson Heights, Downsview North                   0.0   
Bayview Village                                                   0.0   
Bedford Park, Lawrence Manor East                                 0.0   

                                                 Sandwich Place  Restaurant  \
Neighborhood                                                                  
Agincourt                                              0.000000    0.000000   
Alderwood, Long Branch                                 0.142857    0.000000   
Bathurst Manor, Wilson Heights, Downsview North        0.050000    0.050000   
Bayview Village                                        0.000000    0.000000   
Bedford Park, Lawrence Manor East                      0.083333    0.041667   

                                                 Intersection  Bakery  \
Neighborhood                                                            
Agincourt                                                 0.0     0.0   
Alderwood, Long Branch                                    0.0     0.0   
Bathurst Manor, Wilson Heights, Downsview North           0.0     0.0   
Bayview Village                                           0.0     0.0   
Bedford Park, Lawrence Manor East                         0.0     0.0   

                                                 Convenience Store  
Neighborhood                                                        
Agincourt                                                      0.0  
Alderwood, Long Branch                                         0.0  
Bathurst Manor, Wilson Heights, Downsview North                0.0  
Bayview Village                                                0.0  
Bedford Park, Lawrence Manor East                              0.0

In [208]:
#process data for clustering - use standard scaler
from sklearn.preprocessing import StandardScaler
X = toronto_top_venues.values
X = np.nan_to_num(X)
clus_data = StandardScaler().fit_transform(X)



I will cluster the data using Kmeans clustering

In [191]:
#import Kmeans
from sklearn.cluster import KMeans


In [192]:
#start modeling
n_clust = 4
X = clus_data
kmeans = KMeans(init = "k-means++", n_clusters=n_clust, n_init=20)
kmeans.fit(X)
labels = kmeans.labels_
print(labels)

[0 3 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 3 0 0 0 0 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 3 0 0
 3 0 0 0 3 0 0 0 0 0 0 0 3 0 3 0 0 0 0 1]


In [193]:
#add the cluster labels back to the dataframe and group by cluster to look at differences in clusters
toronto_top_venues["Clus_km"] = labels
toronto_top_venues.groupby("Clus_km").mean()

Park  Coffee Shop  Pizza Place      Café  Fast Food Restaurant  \
Clus_km                                                                       
0        0.068641     0.061535     0.020130  0.036095              0.011867   
1        0.200000     0.000000     0.100000  0.000000              0.000000   
2        0.000000     0.000000     0.000000  0.000000              1.000000   
3        0.020607     0.055386     0.121087  0.021689              0.026630   

         Sandwich Place  Restaurant  Intersection    Bakery  Convenience Store  
Clus_km                                                                         
0              0.008452    0.023722      0.010164  0.021385           0.005229  
1              0.000000    0.000000      0.233333  0.000000           0.233333  
2              0.000000    0.000000      0.000000  0.000000           0.000000  
3              0.137533    0.024427      0.012987  0.012987           0.018182

In [194]:
#how many venues are there per cluster?
toronto_top_venues.groupby("Clus_km").count()

Park  Coffee Shop  Pizza Place  Café  Fast Food Restaurant  \
Clus_km                                                               
0          77           77           77    77                    77   
1           5            5            5     5                     5   
2           1            1            1     1                     1   
3          11           11           11    11                    11   

         Sandwich Place  Restaurant  Intersection  Bakery  Convenience Store  
Clus_km                                                                       
0                    77          77            77      77                 77  
1                     5           5             5       5                  5  
2                     1           1             1       1                  1  
3                    11          11            11      11                 11

## Analysis

I have separated the neighborhoods into 4 clusters. 

**Cluster 0** has 77 neighborhoods
**Cluster 1** has 5 neighborhoods
**Cluster 2** has 1 neighborhood
**Cluster 3** has 11 neighborhoods

It seems that based on this analysis, each cluster has distinctive characteristics. 

**Cluster 0** is characterized by having a high frequency of many types of venues including parks, restaurants, cafes, etc. This is likely to be neighborhoods where people live. We could subset by these neighborhoods to find ones most similar to the ones we like. 

**Cluster 1** is characterized by having a high frequency of parks, intersections, and convenience stores. These neighborhoods are likely close to nature or in more industrial districts. I would have to do more analysis to identify which was which

**Cluster 2** has only 1 member neighborhood and has only fast food restaurant venues. This is probably not a place you would want to live.

**Cluster 3** like Cluster 0 has a variety of restaurants and venues, but it is most characterized by high frequency of coffee shops and sandwich places. These neighborhoods may be closer to districts that cater to the breakfast and lunch crowd. 

___

These neighborhoods are mapped below


In [203]:
#add cluster to neighborhoods
neighborhoods = neighborhoods[neighborhoods['Neighbourhood'].isin(list(toronto_top_venues.index))]
neighborhoods['Cluster'] = [toronto_top_venues.loc[n,'Clus_km'] for n in neighborhoods['Neighbourhood']]
neighborhoods.head()

Postal Code           Borough                                Neighbourhood  \
2         M3A        North York                                    Parkwoods   
3         M4A        North York                             Victoria Village   
4         M5A  Downtown Toronto                    Regent Park, Harbourfront   
5         M6A        North York             Lawrence Manor, Lawrence Heights   
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  Cluster  
2  43.753259 -79.329656        0  
3  43.725882 -79.315572        0  
4  43.654260 -79.360636        0  
5  43.718518 -79.464763        0  
6  43.662301 -79.389494        0

In [204]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(n_clust)
ys = [i + x + (i*x)**2 for i in range(n_clust)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighbourhood'], neighborhoods['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters